<a href="https://colab.research.google.com/github/danny980521/BOAZ_Analysis_Mentoring_A/blob/main/BOAZ_%EB%A9%98%EB%A9%98_%EA%B2%B0%EC%A0%95%ED%8A%B8%EB%A6%AC%26%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#위스콘신 유방암 데이터 로드

In [27]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
data = cancer.data
label = cancer.target

data_df = pd.DataFrame(data, columns = cancer.feature_names)
data_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


#decision tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dt_clf = DecisionTreeClassifier(random_state=123)

scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.915603


#boosting

##adaboost

In [3]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(n_estimators=200, random_state=123)

scores = cross_val_score(ada_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.975423


##xgboost

In [4]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=200, random_state=123)

scores = cross_val_score(xgb_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.973638


##lightgbm

In [5]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=300, random_state=123)

scores = cross_val_score(lgbm_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.971883


#bagging

In [6]:
#코드 구동에 약 10초가 소요됩니다.
from sklearn.ensemble import BaggingClassifier 

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16, random_state=123), 
    n_estimators=200, max_samples=1.0, bootstrap=True, n_jobs=-1, random_state=123)

scores = cross_val_score(bag_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.963111


##random forest
decision tree에 배깅을 적용한 것이 random forest이므로 위의 코드와 성능이 비슷할 것으로 예측된다.

In [7]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=200, max_leaf_nodes=16, n_jobs=-1, random_state=123)

scores = cross_val_score(rf_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.959618


##배깅 직접 구현
배깅의 직접 구현도 어렵지 않다. 하는 김에 소프트보팅 방법도 직접 구현해보자.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 123)

bagging_pred = [] #각 모델들의 레이블 예측
bagging_predict_proba = [] #각 모델들의 레이블에 속할 확률 예측
bagging_predict_result = [] #test set으로 평가한 각 모델들의 accuracy

for _ in range(10):  
    # 반복 복원추출과정. 중복을 허용하지 않음으로 인해 페이스팅을 구현하고자 한다면 np.random.choice에서 replace=False로 설정하면 된다. default 값은 True이다.
    data_index = [data_index for data_index in range(X_train.shape[0])]
    random_data_index = np.random.choice(data_index, X_train.shape[0]) 
    
    # decision tree
    bg_X_train = pd.DataFrame(X_train).iloc[random_data_index,]
    bg_y_train = pd.DataFrame(y_train).iloc[random_data_index,]
    dt_clf.fit(bg_X_train, bg_y_train)

    pred = dt_clf.predict(X_test)
    bagging_pred.append(dt_clf.predict(X_test))
    bagging_predict_proba.append(dt_clf.predict_proba(X_test))
    bagging_predict_result.append(accuracy_score(y_test, pred))

In [9]:
#모델들 예측 결과
bagging_predict_result

[0.9035087719298246,
 0.956140350877193,
 0.9473684210526315,
 0.9385964912280702,
 0.868421052631579,
 0.9035087719298246,
 0.956140350877193,
 0.9210526315789473,
 0.9736842105263158,
 0.956140350877193]

In [10]:
# 최종 결과. 각 모델들의 예측 확률을 소프트 보팅 방법으로 합한 후 대소비교를 통해 최종 레이블을 0 또는 1로 결정해주었다. 
last_proba = np.zeros_like(bagging_predict_proba[0])
for i in range(10) :
    last_proba += bagging_predict_proba[i]
last_pred = np.zeros_like(bagging_pred[0])
for i in range(len(last_proba)) :
    last_pred[i] = 0 if last_proba[i][0] >= last_proba[i][1] else 1
  
print(accuracy_score(y_test, last_pred))

0.9736842105263158


부스팅 모델들에 배깅을 적용할 수는 없을까?
##bagged xgboost

In [31]:
#코드 구동에 약 2분이 소요됩니다.
bag_xgb_clf = BaggingClassifier(
    XGBClassifier(max_features = "auto", n_estimators=200, random_state=123), 
    n_estimators= 200, max_samples=1.0, bootstrap=True, n_jobs=-1, random_state=123)

scores = cross_val_score(bag_xgb_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.96662


##bagged lightgbm

In [32]:
#코드 구동에 약 4분이 소요됩니다.
bag_lgbm_clf = BaggingClassifier(
    LGBMClassifier(max_features = "auto", n_estimators=300, random_state=123), 
    n_estimators= 200, max_samples=1.0, bootstrap=True, n_jobs=-1, random_state=123)

scores = cross_val_score(bag_lgbm_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.973638


#voting

##hard voting

In [13]:
from sklearn.ensemble import VotingClassifier

vo_clf1 = VotingClassifier(estimators = [('ada',ada_clf), ('xgb',xgb_clf)], voting = 'hard')

scores = cross_val_score(vo_clf1, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.975408


##soft voting

In [14]:
vo_clf2 = VotingClassifier(estimators = [('ada',ada_clf), ('xgb',xgb_clf)], voting = 'soft')

scores = cross_val_score(vo_clf2, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.973638


사이킷런은 weighted average를 활용한 soft voting 또한 지원한다. 보통 점수가 더 잘 나오는 쪽에 가중치를 크게 주면 높은 점수를 기대할 수 있다. 그러나 cv셋에 따라 오버피팅의 가능성이 있으니 주의하자. 

In [15]:
vo_clf3 = VotingClassifier(estimators = [('ada',ada_clf), ('xgb',xgb_clf)], voting = 'soft', weights = [0.7, 0.3])

scores = cross_val_score(vo_clf3, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.975392


분류기간의 성능 차이가 크면 아래처럼 좋은 앙상블 결과를 기대하기 힘들다. 

In [16]:
vo_clf4 = VotingClassifier(estimators = [('tree',dt_clf), ('xgb',xgb_clf)], voting = 'soft')

scores = cross_val_score(vo_clf4, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.934995


3개 이상의 분류기의 앙상블도 물론 가능하다.

In [17]:
vo_clf5 = VotingClassifier(estimators = [('ada',ada_clf), ('xgb',xgb_clf), ('lgbm', lgbm_clf)], voting = 'hard')

scores = cross_val_score(vo_clf5, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.971883


#stacking
핸즈온 머신러닝에서는 사이킷런이 스태킹을 직접 지원하지 않는다고 하지만, 업데이트가 된 것인지 찾아보니 지원하는 모듈이 있었다. 심지어 default로 stratified 5-fold를 이용하여 테스트 셋을 예측한다.

In [18]:
from sklearn.ensemble import StackingClassifier

st_clf = StackingClassifier(estimators=[('ada',ada_clf), ('xgb',xgb_clf)])

scores = cross_val_score(st_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.973638


final_estimator를 다른 분류기로 설정할 수 있다. default는 linear regressor이다.
그 외에 다양한 튜닝법은 사이킷런 레퍼런스에 자세히 설명되어 있다.
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html?highlight=stacking#sklearn.ensemble.StackingClassifier

In [19]:
#코드 구동에 약 1분이 소요됩니다.
st_clf = StackingClassifier(estimators=[('ada',ada_clf), ('xgb',xgb_clf), ('rf', rf_clf)], final_estimator=LGBMClassifier())

scores = cross_val_score(st_clf, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 6))

cv score :  0.971914


##stacking 직접 구현(1)
스태킹 직접 구현 또한 어렵지 않으니 해보도록 하자. 파머완 코드를 참고했다.

In [20]:
from sklearn.linear_model import LogisticRegression

lr_final = LogisticRegression(C=10, random_state=123)

dt_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)
xgb_clf.fit(X_train, y_train)

dt_pred = dt_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
xgb_pred = xgb_clf.predict(X_test)

print('dt 정확도 :',accuracy_score(y_test, dt_pred))
print('rf 정확도 :',accuracy_score(y_test, rf_pred))
print('ada 정확도 :',accuracy_score(y_test, ada_pred))
print('xgb 정확도 :',accuracy_score(y_test, xgb_pred))

dt 정확도 : 0.9298245614035088
rf 정확도 : 0.9912280701754386
ada 정확도 : 0.9649122807017544
xgb 정확도 : 0.9736842105263158


In [21]:
#각 예측값을 하나의 배열에 담는다.
stacked_pred = np.array([dt_pred, rf_pred, ada_pred, xgb_pred])
print(stacked_pred.shape)

# transpose를 이용해 행과 열의 위치를 교환한다.
stacked_pred = np.transpose(stacked_pred)
print(stacked_pred.shape)

(4, 114)
(114, 4)


In [22]:
lr_final.fit(stacked_pred, y_test)
final_pred = lr_final.predict(stacked_pred)

print('최종 메타 모델 정확도 : ',accuracy_score(y_test, final_pred))

최종 메타 모델 정확도 :  0.9912280701754386


위의 스태킹 모델은 y_test를 학습했기에 과적합 문제가 발생할 수 있다. (랜덤 시드에 따라 정확도가 1.0이 되기도 한다.)
##stacking 직접 구현(2)
cv세트 기반의 신뢰할만한 스태킹 모델을 직접 구현하자. 파머완에서는 kfold를 사용했지만 여기서는 stratifiedkfold를 이용해 구현하겠다.

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds 값으로 KFold 생성
    kf = StratifiedKFold(n_splits=n_folds, shuffle=False, random_state=123)
    
    # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__,' model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n, y_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트: ',folder_counter+1,' 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어지 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [24]:
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 5)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 5)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_clf, X_train, y_train, X_test, 5)

DecisionTreeClassifier  model 시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
RandomForestClassifier  model 시작
	 폴드 세트:  1  시작


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


AdaBoostClassifier  model 시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


XGBClassifier  model 시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작


In [25]:
Stack_final_X_train = np.concatenate((dt_train, rf_train, ada_train, xgb_train), axis=1)
Stack_final_X_test = np.concatenate((dt_test, rf_test, ada_test, xgb_test), axis=1)
print('원본 학습 피처 데이터 shape:', X_train.shape, '원본 테스트 피처 shape:',X_test.shape)
print('스태킹 학습 피처 데이터 shape:',Stack_final_X_train.shape,
     '스태킹 테스트 피처 데이터 shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 shape: (455, 30) 원본 테스트 피처 shape: (114, 30)
스태킹 학습 피처 데이터 shape: (455, 4) 스태킹 테스트 피처 데이터 shape: (114, 4)


In [26]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: ', accuracy_score(y_test, stack_final))

최종 메타 모델의 예측 정확도:  0.9824561403508771


#과제 : 천하제일 오버피팅 대회
데이터는 위스콘신 유방암 데이터이고, 평가지표는 위의 코드들과 같이 같이 stratified 5-fold cv score입니다. 점수를 위해서라면 어떤 코드라도 작성하셔도 됩니다. 여러분의 하이퍼 패러미터 튜닝 / 앙상블 실력을 보여주세요.

In [ ]:
#코드
#--------------------------------------------------------------------










#--------------------------------------------------------------------
#점수 산정만 가능하다면 밑의 두 줄 또한 얼마든지 바꾸셔도 됩니다.
scores = cross_val_score(당신의 분류기, data, label, scoring = 'accuracy', cv = 5)
print('cv score : ', np.round(np.mean(scores), 8))